# Clone and Compare Workflow

This notebook demonstrates the **CLB Engineering LLM Forward Approach** to QAQC:

- Non-destructive cloning of basin, met, and run configurations
- Side-by-side comparison in HEC-HMS GUI
- Parallel execution of baseline vs. updated scenarios

## Why Clone?

Cloning allows you to:
1. **Preserve original baseline models** - No risk of losing original work
2. **Test modifications safely** - Experiment without consequences
3. **Compare results side-by-side** - Visual GUI comparison
4. **Maintain traceability** - Clone metadata in descriptions
5. **Use separate DSS files** - Clean comparison between scenarios

**Estimated Time**: 20-30 minutes (including ~1-2 minutes for HMS execution)

In [1]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

In [2]:
from pathlib import Path
from hms_commander import (
    HmsExamples,
    init_hms_project,
    HmsBasin,
    HmsMet,
    HmsRun,
    HmsCmdr,
    HmsResults,
    HmsDss
)

print("hms-commander loaded")

hms-commander loaded


## 1. Initialize Project and Extract Example

In [3]:
# Extract HMS example project
project_path = HmsExamples.extract_project(
    "castro",
    output_path=Path.cwd() / 'hms_example_projects' / 'castro_clone_workflow'
)
print(f"Project extracted to: {project_path}")

# Initialize project
hms = init_hms_project(project_path)

# View available components
print("\nBasin Models:")
print(hms.basin_df[['name', 'total_area', 'loss_methods']])
print("\nMet Models:")
print(hms.met_df[['name', 'precip_method']])
print("\nRuns:")
print(hms.run_df[['name', 'basin_model', 'met_model', 'dss_file']])

2026-01-08 13:24:56 - hms_commander.HmsExamples - INFO - Found HMS 4.10 at C:\Program Files\HEC\HEC-HMS\4.10


2026-01-08 13:24:56 - hms_commander.HmsExamples - INFO - Found HMS 4.11 at C:\Program Files\HEC\HEC-HMS\4.11


2026-01-08 13:24:56 - hms_commander.HmsExamples - INFO - Found HMS 4.12 at C:\Program Files\HEC\HEC-HMS\4.12


2026-01-08 13:24:56 - hms_commander.HmsExamples - INFO - Found HMS 4.13 at C:\Program Files\HEC\HEC-HMS\4.13


2026-01-08 13:24:56 - hms_commander.HmsExamples - INFO - Found HMS 4.4.1 at C:\Program Files\HEC\HEC-HMS\4.4.1


...

2026-01-08 13:24:56 - hms_commander.HmsPrj - INFO -   Control specs: 1


2026-01-08 13:24:56 - hms_commander.HmsPrj - INFO -   Simulation runs: 2


2026-01-08 13:24:56 - hms_commander.HmsPrj - INFO -   Gages: 2


2026-01-08 13:24:56 - hms_commander.HmsPrj - INFO -   Paired data tables: 1


Project extracted to: C:\GH\hms-commander\examples\hms_example_projects\castro_clone_workflow\castro

Basin Models:
       name  total_area      loss_methods
0  Castro 1       40.51  Initial+Constant
1  Castro 2       49.51  Initial+Constant

Met Models:
      name   precip_method
0  GageWts  Weighted Gages

Runs:
      name basin_model met_model     dss_file
0  Current    Castro 1   GageWts  Current.dss
1   Future    Castro 2   GageWts   Future.dss


## 2. Clone Basin Model

Clone the baseline basin to create an updated version for modification.

In [4]:
# Clone basin for modifications
baseline_basin = hms.list_basin_names()[0]
print(f"Cloning basin: {baseline_basin}")

new_basin_path = HmsBasin.clone_basin(
    template_basin=baseline_basin,
    new_name="Castro_Updated",
    description="Updated parameters for calibration comparison",
    hms_object=hms
)

if new_basin_path:
    print(f"\n[OK] Basin cloned successfully!")
    print(f"New basin file: {new_basin_path}")
    print("New basin will appear in HEC-HMS GUI")

2026-01-08 13:24:56 - hms_commander.HmsUtils - INFO - Cloned file: Castro_1.basin → Castro_Updated.basin


2026-01-08 13:24:56 - hms_commander.HmsUtils - INFO - Added Basin 'Castro_Updated' to project file


2026-01-08 13:24:56 - hms_commander.HmsPrj - INFO - HMS project initialized: castro


2026-01-08 13:24:56 - hms_commander.HmsPrj - INFO -   Version: 4.13


2026-01-08 13:24:56 - hms_commander.HmsPrj - INFO -   Basin models: 2


...

2026-01-08 13:24:56 - hms_commander.HmsPrj - INFO -   Gages: 2


2026-01-08 13:24:56 - hms_commander.HmsPrj - INFO -   Paired data tables: 1


2026-01-08 13:24:56 - hms_commander.HmsBasin - INFO - Re-initialized project to register new basin 'Castro_Updated'


2026-01-08 13:24:56 - hms_commander.HmsBasin - INFO - Cloned basin: Castro 1 → Castro_Updated


Cloning basin: Castro 1

[OK] Basin cloned successfully!
New basin file: C:\GH\hms-commander\examples\hms_example_projects\castro_clone_workflow\castro\Castro_Updated.basin
New basin will appear in HEC-HMS GUI


## 3. Modify Parameters in Cloned Basin

Update parameters in the cloned basin while leaving the original intact.

### Why +5% Imperviousness?

In this example, we increase percent impervious by 5% to simulate:
- **Development scenario**: What if more area becomes paved?
- **Calibration test**: Is the model sensitive to imperviousness?
- **Climate adaptation**: Increased runoff from denser development

This small change (+5%) is:
- **Detectable**: Large enough to see in peak flows
- **Realistic**: Represents moderate development increase
- **Non-destructive**: Easy to compare against baseline

You can adjust this value based on your analysis needs.

In [5]:
# Get the cloned basin file
cloned_basin_file = project_path / "Castro_Updated.basin"

if cloned_basin_file.exists():
    # Get all subbasins
    subbasins = HmsBasin.get_subbasins(str(cloned_basin_file))
    print(f"Found {len(subbasins)} subbasins in cloned basin")
    
    # Show current parameters
    print("\nCurrent subbasin parameters:")
    print("=" * 60)
    for idx, row in subbasins.iterrows():
        subbasin_name = row['name']
        params = HmsBasin.get_loss_parameters(str(cloned_basin_file), subbasin_name)
        print(f"  {subbasin_name}: {params.get('percent_impervious', 'N/A')}% impervious")
else:
    print("Cloned basin file not found - clone may have failed")

2026-01-08 13:24:56 - hms_commander.HmsBasin - INFO - Reading subbasins from: C:\GH\hms-commander\examples\hms_example_projects\castro_clone_workflow\castro\Castro_Updated.basin


2026-01-08 13:24:56 - hms_commander.HmsBasin - INFO - Found 4 subbasins


Found 4 subbasins in cloned basin

Current subbasin parameters:
  Subbasin-3: 10.0% impervious
  Subbasin-4: 15.0% impervious
  Subbasin-1: 2.0% impervious
  Subbasin-2: 8.0% impervious


In [6]:
# Modify parameters (increase imperviousness by 5%)
# This simulates increased development/urbanization
IMPERVIOUSNESS_INCREASE = 5.0  # Configurable parameter

if cloned_basin_file.exists():
    print(f"Updating percent impervious (+{IMPERVIOUSNESS_INCREASE}%):")
    print("=" * 60)
    
    for idx, row in subbasins.iterrows():
        subbasin_name = row['name']
        params = HmsBasin.get_loss_parameters(str(cloned_basin_file), subbasin_name)
        
        if 'percent_impervious' in params and params['percent_impervious'] is not None:
            old_value = params['percent_impervious']
            new_value = min(old_value + IMPERVIOUSNESS_INCREASE, 100.0)  # Cap at 100%
            
            HmsBasin.set_loss_parameters(
                str(cloned_basin_file),
                subbasin_name,
                percent_impervious=new_value
            )
            print(f"  {subbasin_name}: {old_value}% -> {new_value}%")

2026-01-08 13:24:56 - hms_commander.HmsBasin - INFO - Updated loss parameters for subbasin 'Subbasin-3'


2026-01-08 13:24:56 - hms_commander.HmsBasin - INFO - Updated loss parameters for subbasin 'Subbasin-4'


2026-01-08 13:24:57 - hms_commander.HmsBasin - INFO - Updated loss parameters for subbasin 'Subbasin-1'


2026-01-08 13:24:57 - hms_commander.HmsBasin - INFO - Updated loss parameters for subbasin 'Subbasin-2'


Updating percent impervious (+5.0%):
  Subbasin-3: 10.0% -> 15.0%
  Subbasin-4: 15.0% -> 20.0%
  Subbasin-1: 2.0% -> 7.0%
  Subbasin-2: 8.0% -> 13.0%


## 4. Clone Meteorologic Model (Optional)

Clone the met model if you need to modify precipitation inputs.

In [7]:
# Clone met model (optional - only if modifying precipitation)
met_names = hms.list_met_names()
print(f"Available met models: {met_names}")

if met_names:
    new_met_path = HmsMet.clone_met(
        template_met=met_names[0],
        new_name="Castro_Met_Updated",
        description="Cloned met model for comparison",
        hms_object=hms
    )
    print(f"\n[OK] Met model cloned: {new_met_path}")

2026-01-08 13:24:57 - hms_commander.HmsUtils - INFO - Cloned file: GageWts.met → Castro_Met_Updated.met


2026-01-08 13:24:57 - hms_commander.HmsUtils - INFO - Added Met 'Castro_Met_Updated' to project file


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO - HMS project initialized: castro


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Version: 4.13


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Basin models: 2


...

2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Gages: 2


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Paired data tables: 1


2026-01-08 13:24:57 - hms_commander.HmsMet - INFO - Re-initialized project to register new met 'Castro_Met_Updated'


2026-01-08 13:24:57 - hms_commander.HmsMet - INFO - Cloned met: GageWts → Castro_Met_Updated


Available met models: ['GageWts']

[OK] Met model cloned: C:\GH\hms-commander\examples\hms_example_projects\castro_clone_workflow\castro\Castro_Met_Updated.met


## 5. Clone Run Configuration

**Critical for QAQC**: Create a new run with separate DSS output for comparison.

In [8]:
# Clone run with updated components and separate DSS output
run_names = hms.list_run_names()
print(f"Available runs: {run_names}")

if run_names:
    source_run = run_names[0]
    
    success = HmsRun.clone_run(
        source_run=source_run,
        new_run_name="Updated_Scenario",
        new_basin="Castro_Updated",
        new_met="Castro_Met_Updated",  # Use cloned met
        output_dss="updated_scenario.dss",
        description="Increased imperviousness scenario (+5%)",
        hms_object=hms
    )
    
    if success:
        print("\n[OK] Run configuration cloned!")
        print(f"Baseline: {source_run} -> original DSS")
        print("Updated:  Updated_Scenario -> updated_scenario.dss")

2026-01-08 13:24:57 - hms_commander.HmsRun - INFO - Retrieved DSS config for run 'Current': Current.dss


2026-01-08 13:24:57 - hms_commander.HmsRun - INFO - Cloned run: Current → Updated_Scenario


2026-01-08 13:24:57 - hms_commander.HmsRun - INFO -   Basin: Castro_Updated, Met: Castro_Met_Updated, DSS: updated_scenario.dss


2026-01-08 13:24:57 - hms_commander.HmsRun - INFO - Re-initialized project to register new run 'Updated_Scenario'


Available runs: ['Current', 'Future']

[OK] Run configuration cloned!
Baseline: Current -> original DSS
Updated:  Updated_Scenario -> updated_scenario.dss


## 6. Verify Both Run Configurations

Display baseline and cloned run configurations side-by-side.

In [9]:
# Reinitialize to refresh DataFrames
hms = init_hms_project(project_path)

# Find baseline and cloned runs
baseline_runs = [r for r in hms.list_run_names() if 'Updated' not in r and 'Scenario' not in r]
cloned_runs = [r for r in hms.list_run_names() if 'Updated' in r or 'Scenario' in r]

print(f"Baseline runs: {baseline_runs}")
print(f"Cloned runs: {cloned_runs}")

2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO - HMS project initialized: castro


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Version: 4.13


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Basin models: 2


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Met models: 1


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Control specs: 1


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Simulation runs: 3


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Gages: 2


2026-01-08 13:24:57 - hms_commander.HmsPrj - INFO -   Paired data tables: 1


Baseline runs: ['Current', 'Future']
Cloned runs: ['Updated_Scenario']


In [10]:
# Display baseline run configuration
print("Baseline Run Configuration:")
print("=" * 60)
if baseline_runs:
    try:
        baseline_config = HmsRun.get_dss_config(baseline_runs[0], hms_object=hms)
        print(f"  Run Name:   {baseline_runs[0]}")
        print(f"  Basin:      {baseline_config.get('basin_model', 'N/A')}")
        print(f"  Met:        {baseline_config.get('met_model', 'N/A')}")
        print(f"  DSS Output: {baseline_config.get('dss_file', 'N/A')}")
    except Exception as e:
        print(f"  Could not get config: {e}")
else:
    print("  No baseline runs found")

2026-01-08 13:24:57 - hms_commander.HmsRun - INFO - Retrieved DSS config for run 'Current': Current.dss


Baseline Run Configuration:
  Run Name:   Current
  Basin:      Castro 1
  Met:        GageWts
  DSS Output: Current.dss


In [11]:
# Display cloned run configuration
print("Cloned Run Configuration:")
print("=" * 60)
if cloned_runs:
    try:
        cloned_config = HmsRun.get_dss_config(cloned_runs[0], hms_object=hms)
        print(f"  Run Name:   {cloned_runs[0]}")
        print(f"  Basin:      {cloned_config.get('basin_model', 'N/A')}")
        print(f"  Met:        {cloned_config.get('met_model', 'N/A')}")
        print(f"  DSS Output: {cloned_config.get('dss_file', 'N/A')}")
        print(f"  Description: {cloned_config.get('description', 'N/A')}")
    except Exception as e:
        print(f"  Could not get config: {e}")
else:
    print("  No cloned runs found")

2026-01-08 13:24:57 - hms_commander.HmsRun - INFO - Retrieved DSS config for run 'Updated_Scenario': updated_scenario.dss


Cloned Run Configuration:
  Run Name:   Updated_Scenario
  Basin:      Castro_Updated
  Met:        Castro_Met_Updated
  DSS Output: updated_scenario.dss
  Description: Increased imperviousness scenario (+5%)


## 7. Execute Both Runs

Run both baseline and updated scenarios to generate results for comparison.

**Note**: Execution requires HEC-HMS to be installed and properly configured.

In [12]:
# Execute baseline run
if baseline_runs:
    print(f"Executing baseline run: {baseline_runs[0]}")
    try:
        result = HmsCmdr.compute_run(baseline_runs[0], hms_object=hms)
        # Handle both boolean and result object returns
        if hasattr(result, 'success'):
            if result.success:
                print(f"  [OK] Completed in {result.execution_time:.1f}s")
            else:
                print(f"  [FAILED] {result.error_message}")
        elif result:  # Boolean True
            print("  [OK] Execution completed")
        else:
            print("  [FAILED] Execution returned False")
    except Exception as e:
        print(f"  [SKIP] Execution not available: {e}")

2026-01-08 13:24:57 - hms_commander.HmsJython - INFO - Found HEC-HMS: C:\Program Files\HEC\HEC-HMS\4.10


2026-01-08 13:24:57 - hms_commander.HmsCmdr - INFO - Computing run 'Current' in C:\GH\hms-commander\examples\hms_example_projects\castro_clone_workflow\castro


2026-01-08 13:24:57 - hms_commander.HmsJython - INFO - Executing HMS 4.10 via direct Java invocation


2026-01-08 13:24:57 - hms_commander.HmsJython - INFO - Script: C:\GH\hms-commander\examples\hms_example_projects\castro_clone_workflow\castro\hms_script.py


2026-01-08 13:24:57 - hms_commander.HmsJython - INFO - Memory: -Xms128M -Xmx4G


Executing baseline run: Current


2026-01-08 13:24:59 - hms_commander.HmsJython - INFO - HMS 4.10 script executed successfully


2026-01-08 13:24:59 - hms_commander.HmsCmdr - INFO - Run 'Current' completed successfully


  [OK] Execution completed


In [13]:
# Execute cloned run
if cloned_runs:
    print(f"Executing cloned run: {cloned_runs[0]}")
    try:
        result = HmsCmdr.compute_run(cloned_runs[0], hms_object=hms)
        # Handle both boolean and result object returns
        if hasattr(result, 'success'):
            if result.success:
                print(f"  [OK] Completed in {result.execution_time:.1f}s")
            else:
                print(f"  [FAILED] {result.error_message}")
        elif result:  # Boolean True
            print("  [OK] Execution completed")
        else:
            print("  [FAILED] Execution returned False")
    except Exception as e:
        print(f"  [SKIP] Execution not available: {e}")

2026-01-08 13:24:59 - hms_commander.HmsCmdr - INFO - Computing run 'Updated_Scenario' in C:\GH\hms-commander\examples\hms_example_projects\castro_clone_workflow\castro


2026-01-08 13:24:59 - hms_commander.HmsJython - INFO - Executing HMS 4.10 via direct Java invocation


2026-01-08 13:24:59 - hms_commander.HmsJython - INFO - Script: C:\GH\hms-commander\examples\hms_example_projects\castro_clone_workflow\castro\hms_script.py


2026-01-08 13:24:59 - hms_commander.HmsJython - INFO - Memory: -Xms128M -Xmx4G


Executing cloned run: Updated_Scenario


2026-01-08 13:25:01 - hms_commander.HmsJython - INFO - HMS 4.10 script executed successfully


2026-01-08 13:25:01 - hms_commander.HmsCmdr - INFO - Run 'Updated_Scenario' completed successfully


  [OK] Execution completed


## 8. Compare Results

Compare peak flows and hydrographs between baseline and updated scenarios.

**Note**: This section requires DSS files to exist from executed runs.

In [14]:
# Get DSS file paths
baseline_dss = None
updated_dss = project_path / "updated_scenario.dss"

# Determine baseline DSS path from run configuration
if baseline_runs:
    try:
        baseline_config = HmsRun.get_dss_config(baseline_runs[0], hms_object=hms)
        baseline_dss = project_path / baseline_config.get('dss_file', 'castro.dss')
    except Exception:
        baseline_dss = project_path / "castro.dss"

print("=== Results Comparison ===")
print(f"Baseline DSS: {baseline_dss}")
print(f"  Exists: {baseline_dss.exists() if baseline_dss else False}")
print(f"Updated DSS: {updated_dss}")
print(f"  Exists: {updated_dss.exists()}")

2026-01-08 13:25:01 - hms_commander.HmsRun - INFO - Retrieved DSS config for run 'Current': Current.dss


=== Results Comparison ===
Baseline DSS: C:\GH\hms-commander\examples\hms_example_projects\castro_clone_workflow\castro\Current.dss
  Exists: True
Updated DSS: C:\GH\hms-commander\examples\hms_example_projects\castro_clone_workflow\castro\updated_scenario.dss
  Exists: True


In [15]:
# Compare peak flows if both DSS files exist
dss_available = HmsDss.is_available()
baseline_exists = baseline_dss and baseline_dss.exists()
updated_exists = updated_dss.exists()

print(f"DSS library available: {dss_available}")
print(f"Both DSS files exist: {baseline_exists and updated_exists}")

if dss_available and baseline_exists and updated_exists:
    print("\n=== Peak Flow Comparison ===")
    try:
        # Get peak flows from both runs
        baseline_peaks = HmsResults.get_peak_flows(str(baseline_dss))
        print("\nBaseline Peak Flows:")
        print(baseline_peaks)
    except Exception as e:
        print(f"Could not read baseline peaks: {e}")
    
    try:
        updated_peaks = HmsResults.get_peak_flows(str(updated_dss))
        print("\nUpdated Peak Flows:")
        print(updated_peaks)
    except Exception as e:
        print(f"Could not read updated peaks: {e}")
else:
    print("\n[INFO] DSS comparison not available.")
    if not dss_available:
        print("  - DSS library (pyjnius/Java) not configured")
    if not baseline_exists:
        print("  - Baseline DSS file does not exist")
    if not updated_exists:
        print("  - Updated DSS file does not exist")
    print("\nRun HEC-HMS simulations first to generate DSS output files.")

2026-01-08 13:25:01 - hms_commander.dss.core - INFO - Configured JVM with max memory: 4G


DSS library available: True
Both DSS files exist: True

=== Peak Flow Comparison ===
Configuring Java VM for DSS operations...
  Found Java: C:\Program Files\Java\jre1.8.0_471
[OK] Java VM configured


2026-01-08 13:25:01 - hms_commander.dss.hms_dss - INFO - Extracting peaks from 9 paths...


2026-01-08 13:25:01 - hms_commander.dss.hms_dss - INFO - Batch 1/1: processing 9 paths...


2026-01-08 13:25:02 - hms_commander.dss.hms_dss - INFO - Extracted peak flows for 9 elements


2026-01-08 13:25:02 - hms_commander.dss.hms_dss - INFO - No flow paths found in DSS file



Baseline Peak Flows:
       element   peak_flow           peak_time units  \
0       OUTLET  540.274905 1973-01-16 06:55:00   CFS   
1   SUBBASIN-3  309.107130 1973-01-16 06:55:00   CFS   
2  EAST BRANCH  304.284449 1973-01-16 07:20:00   CFS   
3  WEST BRANCH  243.014510 1973-01-16 06:50:00   CFS   
4   SUBBASIN-2  171.422507 1973-01-16 06:55:00   CFS   
5   SUBBASIN-1  162.289258 1973-01-16 06:55:00   CFS   
6      REACH-2  161.391023 1973-01-16 11:20:00   CFS   
7      REACH-1  153.591861 1973-01-16 07:30:00   CFS   
8   SUBBASIN-4  121.793347 1973-01-16 06:50:00   CFS   

                                         dss_path  
0       //OUTLET/FLOW/16JAN1973/5MIN/RUN:CURRENT/  
1   //SUBBASIN-3/FLOW/16JAN1973/5MIN/RUN:CURRENT/  
2  //EAST BRANCH/FLOW/16JAN1973/5MIN/RUN:CURRENT/  
3  //WEST BRANCH/FLOW/16JAN1973/5MIN/RUN:CURRENT/  
4   //SUBBASIN-2/FLOW/16JAN1973/5MIN/RUN:CURRENT/  
5   //SUBBASIN-1/FLOW/16JAN1973/5MIN/RUN:CURRENT/  
6      //REACH-2/FLOW/16JAN1973/5MIN/RUN:CURRENT/  
7

## 9. Visualize Comparison (Optional)

Generate hydrograph overlay if DSS files and matplotlib are available.

### Known Issue: TypeError in DSS Read

Some DSS time series may cause a `TypeError: len() of unsized object` error when reading.
This typically indicates empty or malformed DSS data. The code below handles this gracefully.

In [16]:
# Visualization of results (requires matplotlib and DSS files)
try:
    import matplotlib.pyplot as plt
    matplotlib_available = True
except ImportError:
    matplotlib_available = False
    print("[INFO] matplotlib not installed - visualization skipped")

if matplotlib_available and dss_available and baseline_exists and updated_exists:
    print("Generating hydrograph comparison...")
    try:
        # Get catalog to find flow paths
        baseline_catalog = HmsDss.get_catalog(str(baseline_dss))
        flow_paths = [p for p in baseline_catalog if 'FLOW' in p.upper() and 'OBSERVED' not in p.upper()]
        
        if flow_paths:
            # Read first flow path from both DSS files
            pathname = flow_paths[0]
            print(f"Reading pathname: {pathname}")
            
            try:
                baseline_ts = HmsDss.read_timeseries(str(baseline_dss), pathname)
                updated_ts = HmsDss.read_timeseries(str(updated_dss), pathname)
                
                # Plot comparison
                fig, ax = plt.subplots(figsize=(12, 6))
                ax.plot(baseline_ts.index, baseline_ts['value'], label='Baseline', linewidth=2)
                ax.plot(updated_ts.index, updated_ts['value'], label='Updated (+5% Imp.)', 
                         linewidth=2, linestyle='--')
                ax.set_xlabel('Time')
                ax.set_ylabel('Flow (cfs)')
                ax.set_title('Hydrograph Comparison: Baseline vs Updated')
                ax.legend()
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
            except TypeError as e:
                # Handle the "len() of unsized object" error from DSS read
                print(f"[WARNING] Could not read DSS time series: {e}")
                print("  This may indicate empty or malformed DSS data.")
            except Exception as e:
                print(f"[WARNING] Could not plot hydrographs: {e}")
        else:
            print("[INFO] No FLOW paths found in DSS catalog")
    except Exception as e:
        print(f"[WARNING] Could not generate visualization: {e}")
elif matplotlib_available:
    print("[INFO] Visualization skipped - DSS files not available")
    print("Run HEC-HMS simulations to generate results for comparison.")

Generating hydrograph comparison...
Reading pathname: //SUBBASIN-2/FLOW-UNIT GRAPH/TS-PATTERN/5MIN/RUN:CURRENT/
[WARNING] Could not read DSS time series: len() of unsized object
  This may indicate empty or malformed DSS data.


## 10. Verify in HEC-HMS GUI

After running this notebook, open the project in HEC-HMS GUI:

1. You should see:
   - Original basin: `Castro 1`
   - Cloned basin: `Castro_Updated`
   - Original run: `Current`
   - Cloned run: `Updated_Scenario`

2. Compare results:
   - Open both result plots side-by-side
   - Check descriptions for clone metadata
   - Verify parameter changes in basin editor

## 11. Cleaning Up Cloned Components (Optional)

If you want to remove the cloned components after your analysis, here's how:

### Option 1: Re-extract the Project (Easiest)

```python
import shutil

# Delete and re-extract
shutil.rmtree('hms_example_projects/castro_clone_workflow', ignore_errors=True)
HmsExamples.extract_project('castro', output_path='hms_example_projects/castro_clone_workflow')
```

### Option 2: Delete Specific Files

```python
# Delete cloned files manually
from pathlib import Path

cloned_files = [
    'Castro_Updated.basin',
    'Castro_Met_Updated.met',
    'updated_scenario.dss'
]

for f in cloned_files:
    path = project_path / f
    if path.exists():
        path.unlink()
        print(f"Deleted: {f}")

# Note: You'll also need to remove the run from the .run file
# or HMS will show an error for missing components
```

### Option 3: Use HEC-HMS GUI

Delete cloned components directly in the HEC-HMS interface:
1. Open project in HEC-HMS
2. Right-click on cloned basin/met/run
3. Select "Delete"
4. Save project

## Summary

This workflow demonstrates the **CLB Engineering LLM Forward** approach:

| Principle | Implementation |
|-----------|---------------|
| **Non-destructive** | Original models preserved via cloning |
| **Traceable** | Clone metadata in descriptions |
| **GUI-verifiable** | Components visible in HEC-HMS |
| **Efficient** | Parallel execution capability |
| **Clean comparison** | Separate DSS outputs |

**Perfect for**:
- Parameter sensitivity analysis
- Model updates and calibration
- QAQC workflows
- Scenario comparison

## Next Steps

- **06_results_dss.ipynb**: Deep dive into DSS operations and results extraction
- **07_execution_jython.ipynb**: Advanced execution patterns
- **04_run_management.ipynb**: More details on run configuration